Exporting XML files for Ross Travel 389

In [2]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

with ZipFile ("Fares Data/Ross Travel_389/remote_dataset_11006_2023-07-06_04-02-50.zip", 'r') as zObject:

    zObject.extractall(path="Fares Data\Ross Travel_389")


Initial Parsing Experiment

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root


def extract_data(root):
    data = []
    for record in root.findall('ScheduledStopPoint'):
        row = {}
        for field in record:
            row[field.tag] = field.text
        data.append(row)
    return data

def to_dataframe(data):
    df = pd.DataFrame(data)
    return df

def tag_attrib(root):
    data = []
    for x in root:
        y = x.tag
        data.append(y)
    return data

def Description(root):
    data = []
    for x in root:
        y = x.text
        data.append(y)
    return data


def main(xml_file):
    root = parse_xml(xml_file)
    print(root)
    data = extract_data(root)
    print(data)
    df = to_dataframe(data)
    print(df)
    ta = tag_attrib(root)
    print(ta)
    data_test = Description(root)
    print(data_test)   

if __name__ == "__main__":
    main('Fares Data/Ross Travel_389/ROSS_144_Inbound_ADsgl_43ad4e3d-65a5-49b7-8a29-3dceda1d9905_638241799544014471.xml')



Creating a new file based on the Fares Data directory for saving all JSON files

In [11]:
import shutil, os

def ig_f(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir, f))]

SRC = 'Fares Data'

DEST = 'Fares Data JSON'

shutil.copytree(SRC, DEST, ignore=ig_f)

'Fares Data JSON'

Extract all the xml from their zip files in 'Fare Data' folder and remove an .zip files afterwards

In [19]:
import os, zipfile

my_dir = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Fares Data'

for subdir, dirs, files in os.walk(my_dir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".zip"):
            zip_ref = zipfile.ZipFile(filepath) # create zipfile object
            zip_ref.extractall(subdir) # extract file to dir
            zip_ref.close() # close file
            os.remove(filepath) # delete zipped file

 - Scrape through the Fares Data Files after they've been converted from .zip to .xml
 - Check and see if there are any folders that have subfolders
 - grab a list of these folders
 - move any .xml files from these subfolders to the parent folder
 - delete the subfolders
 - Fares Data folder now ready for JSON conversion

In [ ]:
from os.path import join
from os import listdir, rmdir
from shutil import move
import pathlib

rootdir = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Fares Data/'


def listdirs(rootdir):

    list_of_subdir = []

    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        for file2 in os.listdir(d):
            e = os.path.join(d, file2)
            if os.path.isdir(e):
                list_of_subdir.append(e)
                #listdirs(e)
    return list_of_subdir

def move_xml(sublist):

    for i in sublist:
        path = pathlib.Path(str(i))
        dest = path.parent
        for filename in os.listdir(path):
            new_src = str(path) + os.sep + str(filename)
            new_dst = str(dest)
            print(new_dst)
            if new_src.endswith(".xml"):
                move(new_src, new_dst)

def del_subs(sublist):

    for i in sublist:
        os.rmdir(i)

 
sublist = listdirs(rootdir)
print(sublist)
# move_xml(sublist)
del_subs(sublist)

XML to JSON testing

In [ ]:
import xmltodict, json

with open('Fares Data/Ross Travel_389/ROSS_144_Inbound_ADsgl_43ad4e3d-65a5-49b7-8a29-3dceda1d9905_638241799544014471.xml','r') as test_file:
    obj = xmltodict.parse(test_file.read())

x = json.dumps(obj)

In [1]:
import os, xmltodict, json

SRC = 'C:/Users/Ross/Documents/BODS Project/BODS/Ross Travel'

for subdir, dirs, files in os.walk(SRC):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".xml"):
            with open(filepath,'r') as test_file:
                obj = xmltodict.parse(test_file.read())
                jd = json.dumps(obj, ensure_ascii=False, indent=4)
                new_filepath = filepath.replace('Ross Travel', 'Ross Travel JSON')
                final_filepath = new_filepath.replace('.xml', '.json')
                with open(final_filepath, "w", encoding='utf-8') as jf:
                    jf.write(jd)







In [37]:
import os, zipfile

my_dir = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Fares Data JSON'

for subdir, dirs, files in os.walk(my_dir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".json"):
            os.remove(filepath) # delete zipped file

In [4]:
### Quick SubScript to test out timetable data ###

import os, xmltodict, json

SRC = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Arriva Donnington Timetable Data'

for subdir, dirs, files in os.walk(SRC):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".xml"):
            with open(filepath,'r') as test_file:
                obj = xmltodict.parse(test_file.read())
                jd = json.dumps(obj, ensure_ascii=False, indent=4)
                new_filepath = filepath.replace('Arriva Donnington Timetable Data', 'Arriva JSON')
                final_filepath = new_filepath.replace('.xml', '.json')
                jf = open(final_filepath, "w", encoding='utf-8')
                jf.write(jd)
                jf.close()

Inital Python code for scraping through the Timetable JSON and extracting the nesscessary geo data for Michael to use.

Error comes up as some JSON have different schemas, so code breaks when it runs into that error.

Putting on hold for now until we have our meeting with Spencer qbout the best approach

In [4]:
import pandas as pd
import json
import os
import natsort as ns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def dictionaries(json_dict):

    '''
    Function that grabs the appropraite dictionaries to pass through any of the remaining functions

    Dictionary names are not passed through as a parameter for now for testing purposes

    '''

    stop_dict = json_dict['TransXChange']['StopPoints']['AnnotatedStopPointRef']
    route_dict = json_dict['TransXChange']['RouteSections']['RouteSection']

    return stop_dict, route_dict


def stop_conversion(dict):

    '''
    Convert the stop route dictionary within timetable data and convert it to a DF

    '''

    df = pd.DataFrame.from_dict(dict)
    temp_df = pd.json_normalize(df['Location'])
    df = df.drop('Location', axis=1)
    final_df = pd.concat([df, temp_df],axis=1)

    return final_df

def route_conversion(dict):

    '''
    Convert the route dictionary within timetable data and convert it to a DF
    
    '''

    df = pd.DataFrame.from_dict(dict)
    final_route_df = []

    for i in range(0, len(df)):
        x = list(df['RouteLink'].iloc[i])
        temp_df1 = pd.DataFrame.from_dict(x)
        temp_df2 = pd.json_normalize(temp_df1['From'])
        temp_df2.rename(columns={'StopPointRef': 'From'}, inplace=True)
        temp_df3 = pd.json_normalize(temp_df1['To'])
        temp_df3.rename(columns={'StopPointRef': 'To'}, inplace=True)
        temp_df1 = temp_df1.drop(['From', 'To'], axis=1)
        temp_df4 = pd.concat([temp_df1, temp_df2],axis=1)
        final_df = pd.concat([temp_df4, temp_df3],axis=1)
        final_df['Group @id'] = df.loc[i,'@id']
        first_column = final_df.pop('Group @id')
        final_df.insert(0, 'Group @id', first_column)
        final_route_df.append(final_df)
    
    final_route_df = pd.concat(final_route_df).reset_index(drop=True)

    return final_route_df

def df_merge(route_df, stop_df):

    '''
    Merging the DF's a cleaning before exporting to CSV
    
    '''

    temp_df1 = route_df.merge(stop_df, left_on='From', right_on='StopPointRef')
    final_df = temp_df1.merge(stop_df, left_on='To', right_on='StopPointRef')
    final_df = final_df[['Group @id', '@id', 'From', 'CommonName_x', 'Longitude_x', 'Latitude_x', 'To', 'CommonName_y', 'Longitude_y', 'Latitude_y']]
    
    final_df['@id'] = pd.Categorical(final_df['@id'], ordered=True, categories= ns.natsorted(final_df['@id'].unique()))
    final_df = final_df.sort_values('@id')

    final_df.rename(columns={'Group @id': 'Grouped Route IDs',
                             '@id': 'Route ID',
                             'From': 'From ID',
                             'CommonName_x':'From - Common Name',
                             'Longitude_x': 'From - Longitude',
                             'Latitude_x': 'From - Latitude',
                             'To': 'To ID',
                             'CommonName_y':'To - Common Name',
                             'Longitude_y': 'To - Longitude',
                             'Latitude_y': 'To - Latitude',}, inplace=True)

    final_df.reset_index(drop=True)
    
    return final_df
    
    
def main(json_folder):

    
    for filename in os.listdir(json_folder):
        filepath = os.path.join(json_folder, filename)
        print(filepath)
        with open(filepath) as test_json:
            json_dict = json.load(test_json)
            stop_dict, route_dict = dictionaries(json_dict)
            stop_df = stop_conversion(stop_dict)
            route_df = route_conversion(route_dict)
            merged_df = df_merge(route_df, stop_df)
            merged_df.to_csv(os.path.join('C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Arriva Donnington CSV/', (str(filename) + '.csv')))
                        

if __name__ == "__main__":

    main('C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Arriva Donnington JSON/')


KeyboardInterrupt



In [19]:
import os, xmltodict, json

SRC = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Fares Data/Ross Travel_389'


for subdir, dirs, files in os.walk(SRC):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".xml"):
            with open(filepath,'r') as test_file:
                obj = xmltodict.parse(test_file.read())
                jd = json.dumps(obj, ensure_ascii=False, indent=4)
                new_filepath = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Ross Travel JSON/' + file
                final_filepath = new_filepath.replace('.xml', '.json')
                jf = open(final_filepath, "w", encoding='utf-8')
                jf.write(jd)
                jf.close()





In [44]:
d = 'C:/Users/rosshamilton/OneDrive - KPMG/Documents/BODS/Ross Travel JSON/ROSS_144_Inbound_ADsgl_43ad4e3d-65a5-49b7-8a29-3dceda1d9905_638241799544014471.json'

with open(d) as test_json:
    json_dict = json.load(test_json)

x = json_dict['PublicationDelivery']['dataObjects']['CompositeFrame']
df = pd.DataFrame.from_dict(x)

x = df.loc[5]

print(x)

KeyError: 5